In [ ]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import (
    StrOutputParser,
    JsonOutputParser,
    PydanticOutputParser,
)
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()

True

In [2]:
# 1. Define the Gemini model
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

parser = StrOutputParser()

prompt = PromptTemplate.from_template("Explain {topic} in one sentence.")

chain = prompt | model | parser

result = chain.invoke({"topic": "black hole"})
print(result)

A black hole is a region in spacetime with gravity so strong that nothing, not even light, can escape its pull.


In [7]:
# 1. Define the Gemini model
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# 2. Define JSON output parser
parser = JsonOutputParser()

# 3. Create prompt with structured output instructions
template = PromptTemplate(
    template="Give me 2 facts about {topic} \n{format_instruction}",
    input_variables=["topic"],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

# 4. Create the chain using LCEL pipe operator
chain = template | model | parser

# 5. Invoke the chain
result = chain.invoke({"topic": "Baseball"})

# 6. Print result
print(result)

{'baseball_facts': [{'fact': 'The highest recorded batting average in a single MLB season is .424, achieved by Rogers Hornsby in 1924 while playing for the St. Louis Cardinals.'}, {'fact': "The 'Curse of the Bambino' supposedly plagued the Boston Red Sox for 86 years after they sold Babe Ruth to the New York Yankees in 1919, finally ending with their World Series victory in 2004."}]}


In [19]:
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 2 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'baseball'})

print(result)

{'fact_1': 'The highest recorded batting average in a single MLB season is .424, achieved by Rogers Hornsby in 1924.', 'fact_2': 'The baseball used in Major League Baseball has 108 double stitches.'}


In [16]:
class WeatherReport(BaseModel):
    city: str = Field(description="Name of the city")
    temperature_c: float = Field(description="Temperature in Celsius")
    condition: Literal["sunny", "rainy", "cloudy", "snowy"]

parser = PydanticOutputParser(pydantic_object=WeatherReport)

prompt = PromptTemplate(
    template="Provide a weather report for Tokyo in this format:\n{format_instruction}",
    input_variables=[],
    partial_variables={"format_instruction": parser.get_format_instructions()},
)

chain = prompt | model | parser

result = chain.invoke({})
print(result)

city='Tokyo' temperature_c=25.0 condition='sunny'
